In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from statsmodels.tsa.arima.model import ARIMA
import math
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import tree
import pandas as pd
from sklearn.ensemble import IsolationForest

In [41]:
train_df = pd.read_csv('/content/drive/MyDrive/innova/new_train.csv')


train_df['DATETIME'] = pd.to_datetime(train_df['DATETIME']).dt.tz_localize(None)  # Zaman dilimini kaldır

label_encoder = LabelEncoder()
train_df['SERVER'] = label_encoder.fit_transform(train_df['SERVER'])

#label_encoder_cpu = LabelEncoder()
#train_df['CPU'] = label_encoder_cpu.fit_transform(train_df['CPU'])


start_date = pd.to_datetime('2020-05-15')
end_date = pd.to_datetime('2020-05-30')
train_df_1 = train_df.loc[train_df['DATETIME'] < start_date]
test_df_1 = train_df.loc[(train_df['DATETIME'] >= start_date) & (train_df['DATETIME'] <= end_date)]

X_train = train_df_1[['DATETIME', 'SERVER']]
y_train = train_df_1['CPULOAD']
X_test = train_df_1[['DATETIME', 'SERVER']]
y_test = train_df_1['CPULOAD']

X_train['DATETIME'] = pd.to_datetime(X_train['DATETIME'])
X_train['DATETIME'] = X_train['DATETIME'].astype('int64') // 10**9  # UNIX zamanına dönüşümü

X_test['DATETIME'] = pd.to_datetime(X_test['DATETIME'])
X_test['DATETIME'] = X_test['DATETIME'].astype('int64')


<ipython-input-41-700ea74cebd8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['DATETIME'] = pd.to_datetime(X_train['DATETIME'])
<ipython-input-41-700ea74cebd8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['DATETIME'] = X_train['DATETIME'].astype('int64') // 10**9  # UNIX zamanına dönüşümü
<ipython-input-41-700ea74cebd8>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [42]:
test_df = pd.read_csv('/content/drive/MyDrive/innova/test.csv')
label_encoder = LabelEncoder()
test_df['SERVER'] = label_encoder.fit_transform(test_df['SERVER'])

#label_encoder_cpu = LabelEncoder()
#test_df['CPU'] = label_encoder_cpu.fit_transform(test_df['CPU'])

test_df['DATETIME'] = pd.to_datetime(test_df['DATETIME'])
test_df['DATETIME'] = test_df['DATETIME'].astype('int64')

test = test_df[['DATETIME','SERVER']]
test['CPULOAD'] = 0

<ipython-input-42-186bf563c129>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['CPULOAD'] = 0


In [43]:
test

DATETIME  SERVER  CPULOAD
0     1590872400000000000       0        0
1     1590872400000000000       0        0
2     1590872400000000000       0        0
3     1590872400000000000       0        0
4     1590872400000000000       0        0
...                   ...     ...      ...
4603  1590957900000000000       1        0
4604  1590957900000000000       1        0
4605  1590957900000000000       1        0
4606  1590957900000000000       1        0
4607  1590957900000000000       1        0

[4608 rows x 3 columns]

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# CSV dosyasını oku
#df = pd.read_csv('/content/drive/MyDrive/innova/new_train.csv')

# Veri ön işleme
#df['DATETIME'] = pd.to_datetime(df['DATETIME'])
#df['DATE'] = df['DATETIME'].dt.date
train_df_1 = train_df_1[['DATETIME', 'CPULOAD']].rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})
test = test[['DATETIME', 'CPULOAD']].rename(columns={'DATETIME': 'ds', 'CPULOAD': 'y'})


# Verileri eğitim ve test kümelerine ayır
#train, test = train_test_split(df, test_size=0.2, random_state=42)

# Veri ölçeklendirme
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df_1[['y']])
test_scaled = scaler.transform(test[['y']])

# Veriyi LSTM modeline uygun hale getirme
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 24  # Zaman adımları sayısı
X_train, y_train = create_dataset(train_scaled, train_scaled, time_steps)
X_test, y_test = create_dataset(test_scaled, test_scaled, time_steps)

# LSTM modelini oluştur
model = keras.Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğit
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Tahminleri yap
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

# RMSE değerini hesapla
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print('RMSE:', rmse)


Epoch 1/50
311/311 [==============================] - 26s 64ms/step - loss: 0.0148
Epoch 2/50
311/311 [==============================] - 14s 45ms/step - loss: 0.0060
Epoch 3/50
311/311 [==============================] - 14s 45ms/step - loss: 0.0013
Epoch 4/50
311/311 [==============================] - 14s 45ms/step - loss: 0.0011
Epoch 5/50
311/311 [==============================] - 14s 45ms/step - loss: 0.0011
Epoch 6/50
311/311 [==============================] - 14s 45ms/step - loss: 0.0011
Epoch 7/50
311/311 [==============================] - 16s 52ms/step - loss: 0.0010
Epoch 8/50
311/311 [==============================] - 15s 48ms/step - loss: 0.0010
Epoch 9/50
311/311 [==============================] - 14s 45ms/step - loss: 9.6871e-04
Epoch 10/50
311/311 [==============================] - 14s 45ms/step - loss: 9.4967e-04
Epoch 11/50
311/311 [==============================] - 18s 58ms/step - loss: 9.8418e-04
Epoch 12/50
311/311 [==============================] - 14s 45ms/step - lo

In [49]:
y_pred.to_frame()

AttributeError: ignored

In [59]:
y_pred

array([[0.5854042],
       [0.5854042],
       [0.5854042],
       ...,
       [0.5854055],
       [0.5854055],
       [0.5854055]], dtype=float32)

In [60]:
forecast = pd.DataFrame({'index': range(len(y_pred)), 'CPULOAD': y_pred})

ValueError: ignored

In [50]:
def submission(predict,modelname):
    forecast = pd.DataFrame({'index': range(len(predict)), 'CPULOAD': predict})
    forecast.to_csv(f'{modelname}.csv', index=False)

In [51]:
submission(y_pred,"LSTM")

ValueError: ignored